## 通过 Qwen Agent 操作 sqlite 数据库

> 本节参考 Qwen Agent 的官方 MCP 示例：[QwenLM/Qwen-Agent](https://github.com/QwenLM/Qwen-Agent/blob/main/examples/assistant_mcp_sqlite_bot.py)

根据以下步骤，我们实现一个可以对 sqlite 数据库进行 CRUD（增删改查）操作的 Agent。

**1）启动 `Qwen3-0.6B-FP8` 模型服务**

来到项目根目录下，进入 `test_qwen3` 文件夹，启动 vLLM 服务（这需要你事先按 [/test_qwen3/2.vllm.ipynb](/test_qwen3/2.vllm.ipynb) 的方法配置好环境，并下载好模型）：

```bash
cd test_qwen3
python vllm_server.py
```

**2）启动 Qwen-Agent 服务**

将官方的 `assistant_mcp_sqlite_bot.py` 示例拷贝到当前目录。然后修改代码中的 `llm_cfg` 参数：

```python
llm_cfg = {
    'model': 'Qwen3-0.6B-FP8', # 模型id
    'model_server': 'http://localhost:8000/v1', # vllm服务地址
    'api_key': 'token-kcgyrk', # vllm的api_key
    'generate_cfg': {
        'top_p': 0.95,
        'temperature': 0.6,
}
```

然后运行 `python assistant_mcp_sqlite_bot.py`。这份代码做了两件事：

- 在本地启动 `mcp-server-sqlite` 服务，相当于 MCP Server
- 在本地启动 Qwen-Agent 基于 Gradio 开发的简单 WebUI，相当于 MCP Client

**3）通过 WebUI 进行对话**

启动服务需要一点点时间，启动完成后，在浏览器打开 [http://127.0.0.1:7860](http://127.0.0.1:7860)。

你就可以在 WebUI 中通过对话，对数据库进行新建表、插入记录、删除记录等操作了。

> **Note:** 我的对话记录如下：
> 
> 1. 数据库里有几张表
> 2. 创建一个学生表包括学生的姓名、年龄
> 3. 这张表有哪些字段，它的建表语句是怎么样的
> 4. 增加一个学生名字叫韩梅梅，今年6岁
> 5. 这张表里有数据吗？如果有，请取回
> 6. 再为这个数据表添加十条记录，名字年龄随你取
> 7. 请取回目前数据表中的所有记录，并以 markdown 表格的方式呈现

**4）通过代码进行对话**

Qwen-Agent 不仅可以通过 WebUI 访问，也可以通过代码直接访问。下面就不进行完整对话了。仅做演示：

In [1]:
import os
import asyncio
from typing import Optional

from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI


# 在 jupyter 中运行
ROOT_RESOURCE = os.path.join(os.getcwd(), 'resource')

# 在 python 文件中运行
# ROOT_RESOURCE = os.path.join(os.path.dirname(__file__), 'resource')


def init_agent_service():
    llm_cfg = {
        'model': 'Qwen3-0.6B-FP8',
        'model_server': 'http://localhost:8000/v1',
        'api_key': 'token-kcgyrk',
        'generate_cfg': {
            'top_p': 0.95,
            'temperature': 0.6,
        }
    }
    system = ('你扮演一个数据库助手，你具有查询数据库的能力')
    tools = [{
        "mcpServers": {
            "sqlite" : {
                "command": "uvx",
                "args": [
                    "mcp-server-sqlite",
                    "--db-path",
                    "test.db"
                ]
            }
        }
    }]
    bot = Assistant(
        llm=llm_cfg,
        name='数据库助手',
        description='数据库查询',
        system_message=system,
        function_list=tools,
    )

    return bot

In [2]:
# 初始化 Agent
bot = init_agent_service()

2025-05-18 17:26:41,750 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['sqlite']
2025-05-18 17:26:41,767 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: sqlite


不附带文件的对话

In [3]:
query = '数据库里有几张表'
messages = [{'role': 'user', 'content': query}]

# 流式输出
# for response in bot.run(messages):
#     print('bot response:', response)

# 整段输出
response = bot.run_nonstream(messages)
print('bot response:', response)

bot response: [{'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户问数据库里有几张表。我需要先确定用户的问题是什么。他们可能是在询问数据库的结构，或者想了解表的数量。根据提供的工具，有一个sqlite-list_tables函数可以列出所有表。所以，我应该调用这个函数来获取表的数量。不过，用户的问题可能不需要具体的表数量，而是想知道有哪些表。因此，正确的做法是使用sqlite-list_tables函数来列出所有表，这样用户就能知道具体的表信息了。不需要其他工具，因为用户的问题比较直接，只需要列出表即可。\n', 'name': '数据库助手'}, {'role': 'assistant', 'content': '', 'name': '数据库助手', 'function_call': {'name': 'sqlite-list_tables', 'arguments': '{}'}}, {'role': 'function', 'content': "[{'name': 'students'}]", 'name': 'sqlite-list_tables'}, {'role': 'assistant', 'content': '', 'reasoning_content': "\n好的，用户问数据库里有多少张表。我需要先确定如何回答这个问题。根据之前的工具列表，有一个sqlite-list_tables函数可以列出所有表。所以，我应该调用这个函数来获取表名，然后统计数量。\n\n不过，用户可能想知道具体的表数量，而不仅仅是名称。可能需要先列出所有表，再数一数有多少张。但根据工具的描述，sqlite-list_tables函数返回的是表名列表。所以，我需要先执行这个函数，然后从结果中提取表的数量。\n\n假设调用sqlite-list_tables后得到的响应是['students']，那么表的数量是1。但可能用户需要确认是否有其他表，或者需要更详细的信息。不过根据现有工具，只能通过列表来回答表的数量。因此，正确的步骤是调用列表函数，然后回答数量。\n", 'name': '数据库助手'}, {'role': 'assistant', 'content': '\n\n数

In [4]:
print('result:', response[-1]['content'].strip())

result: 数据库中目前有1张表：students。


附带文件作为 context 的对话

In [5]:
query = '`poem.pdf` 文件里有什么'
file = os.path.join(ROOT_RESOURCE, 'poem.pdf')

messages = [{
    'role': 'user',
    'content': [
        {'text': query},
        {'file': file}
    ]
}]

# 整段输出
response = bot.run_nonstream(messages)
print('bot response:', response)

2025-05-18 17:27:03,342 - simple_doc_parser.py - 411 - INFO - Read parsed /mnt/d/GitHub/mcp-project/test_qwen_agent/resource/poem.pdf from cache.
2025-05-18 17:27:03,343 - doc_parser.py - 108 - INFO - Start chunking /mnt/d/GitHub/mcp-project/test_qwen_agent/resource/poem.pdf (poem.pdf)...
2025-05-18 17:27:03,343 - doc_parser.py - 126 - INFO - Finished chunking /mnt/d/GitHub/mcp-project/test_qwen_agent/resource/poem.pdf (poem.pdf). Time spent: 4.9591064453125e-05 seconds.
2025-05-18 17:27:03,370 - base_search.py - 56 - INFO - all tokens: 3081
2025-05-18 17:27:03,372 - base_search.py - 59 - INFO - use full ref


bot response: [{'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，我现在需要处理用户关于上传的poem.pdf文件的问题。首先，用户提到上传了文件，并询问其中的内容。我需要检查提供的知识库内容，看看是否有相关的信息。\n\n查看知识库中的内容，发现有多个段落，每段都是中文的诗句，分别来自不同的作者。例如，孟郊的《雪》、洪升的《雪望》、韩愈的《春雪》等。这些诗句都是关于自然景象，如雪、梅花、春天等主题。看起来用户上传的文件可能包含这些诗歌文本，或者有其他与这些诗歌相关的数据。\n\n用户的问题是关于文件的内容，可能需要列出文件中包含的诗歌或数据。根据知识库中的内容，文件中包含了17到30个诗行，每行都是不同的诗句，涵盖自然景象和情感表达。因此，可以确定文件中包含这些诗句，但需要确认是否还有其他内容，比如数据表或表单信息。\n\n接下来，我需要确定是否需要调用任何工具来获取更多信息。根据提供的工具，SQLite数据库查询、创建表、列出表、描述表和添加洞察力。但当前用户的问题是关于文件的内容，而知识库中并没有直接提到文件中的内容，只是提供了诗行。因此，可能需要指出文件中包含这些诗行，但不确定是否有其他数据。\n\n因此，回答用户时应简要说明文件中包含的诗行，并确认是否需要进一步帮助。同时，检查是否有遗漏的信息，确保回答准确。\n', 'name': '数据库助手'}, {'role': 'assistant', 'content': '\n\n`poem.pdf` 文件中包含以下内容：\n\n1. 17到30首中文古诗，涵盖自然景象（如雪、梅、春雨、秋风）和情感表达（如思乡、离别、思乡愁等），诗句风格各异，展现了中国古代诗歌的多样性。\n\n这些诗行通过不同的作者创作，展现了丰富的文化内涵和审美价值。如果您需要进一步分析或处理这些诗行，可以提供具体需求。', 'name': '数据库助手'}]


In [6]:
print('result:', response[-1]['content'].strip())

result: `poem.pdf` 文件中包含以下内容：

1. 17到30首中文古诗，涵盖自然景象（如雪、梅、春雨、秋风）和情感表达（如思乡、离别、思乡愁等），诗句风格各异，展现了中国古代诗歌的多样性。

这些诗行通过不同的作者创作，展现了丰富的文化内涵和审美价值。如果您需要进一步分析或处理这些诗行，可以提供具体需求。
